# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Attention, Bidirectional, LSTM, Dense, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf

# Data Loading

In [ ]:
X = np.load(r'/content/drive/My Drive/Model Files/X_train.npy')
y = np.load(r'/content/drive/My Drive/Model Files/y_train.npy')

In [ ]:
num_classes = 94
y_one_hot = np.zeros((y.shape[0], y.shape[1], num_classes), dtype=np.int32)
for i in range(y.shape[0]):
    y_one_hot[i] = to_categorical(y[i], num_classes=num_classes)

# Model Architectures

In [ ]:
epochs = 50
batch_size = 32

## Attention Layer

In [ ]:
# LSTM MODEL WITH ATTENTION LAYER

max_sequence_length = 5
feature_vector_size = len(X[0][0]) # should be 2048
model = Sequential()
#reshaped_X = tf.expand_dims(X[:,0], -1)
#model.add(Attention()([reshaped_X, X])) #if self-attention layer precedes lstm
model.add(LSTM(units=100, input_shape=(max_sequence_length, feature_vector_size), return_sequences = True)) # sequence-to-sequence
model.add(Attention()) #if attention is after lstm
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 1D Convolutional Layer

In [ ]:
# LSTM MODEL WITH 1D CONVOLUTION LAYER

max_sequence_length = 5
feature_vector_size = len(X[0][0]) # should be 2048
model = Sequential()
#1D convolution layer # num of filters = 64 , length 3
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(max_sequence_length, feature_vector_size)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=100, return_sequences = True)) # sequence-to-sequence
model.add(Dense(units=num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Multiple LSTMs

In [ ]:
# LSTM MODEL WITH MULTIPLE LSTM

max_sequence_length = 5
feature_vector_size = len(X[0][0]) # should be 2048
model = Sequential()
model.add(LSTM(units=2048, input_shape=(max_sequence_length, feature_vector_size), return_sequences = True)) # sequence-to-sequence
#add more than 1 layer
model.add(LSTM(units=1024, return_sequences = True)) # no need to specify input_shape again
model.add(LSTM(units=512, return_sequences = True)) # units are halved

model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## BiLSTM

In [ ]:
# LSTM MODEL WITH BI-DIRECTIONAL LSTM

max_sequence_length = 5
feature_vector_size = len(X[0][0]) # should be 2048
model = Sequential()
model.add(Bidirectional(LSTM(units=2048, input_shape=(max_sequence_length, feature_vector_size), return_sequences = True)))
model.add(TimeDistributed(Dense(units=num_classes, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Training

In [ ]:
model.fit(X, y_one_hot, epochs=epochs, batch_size=batch_size, validation_split=0.2)

In [ ]:
model.save(r'/content/drive/My Drive/Model Files/Full_Pad_Model_50e_2048units_2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Train and Test Accuracies

In [ ]:
train_results = model.evaluate(X, y_one_hot)
print("Train Loss:", train_results[0])
print("Train Accuracy:", train_results[1])

229/229 [==============================] - 82s 356ms/step - loss: 1.0115 - accuracy: 0.8680
Train Loss: 1.0115270614624023
Train Accuracy: 0.868045449256897


In [ ]:
X_test = np.load(r'/content/drive/My Drive/Model Files/X_test.npy')
y_test = np.load(r'/content/drive/My Drive/Model Files/y_test.npy')

In [ ]:
y_test_ohe = np.zeros((y_test.shape[0], y_test.shape[1], num_classes), dtype=np.int32)
for i in range(y_test.shape[0]):
    y_test_ohe[i] = to_categorical(y_test[i], num_classes=num_classes)

In [ ]:
test_results = model.evaluate(X_test, y_test_ohe)
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])

58/58 [==============================] - 21s 356ms/step - loss: 1.1969 - accuracy: 0.8289
Test Loss: 1.1969400644302368
Test Accuracy: 0.8288816213607788
